# DPO

In [1]:
!pip install transformers accelerate hf-transfer peft -Uq

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM  # 토크나이저, 언어모델 로드

c:\Users\Playdata\nlp\nlp_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# policy model (dpo 학습대상)
# ref model (참조모델, 학습 대상아님) : 비교대상
model_id = 'LGAI-EXAONE/EXAONE-4.0-1.2B'
# 정책(학습) 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.bfloat16,  # 메모리/속도 최적화
    device_map='auto',     # CPU/GPU 자동 분산
)
tokenizer = AutoTokenizer.from_pretrained(model_id)  # 동일한 모델의 토크나이저 로드

Loading weights: 100%|██████████| 332/332 [00:00<00:00, 701.75it/s, Materializing param=model.norm.weight]                                


In [4]:
# 샘플데이터
prompt = '파이썬이 뭐야?'
chosen = '파이썬은 배우기 쉽고, 강력한 프로그래밍 언어입니다.'
rejected = '파이썬은 뱀이지~'

In [7]:
# 생성확률 계산
# - 모델이 답변을 실제 생성하는게 아니라
# - 데이터셋에서 주어진 텍스트(토큰)에 대한 생성확률을 계산하고,
# - chosen데이터의 생성확률은 높히고, rejected데이터의 생성확률은 낮추는 학습

# 모델이 "이 답변이 얼마나 그럴듯한지"를 점수(log-prob 합)로 계산하는 함수
def get_logprob(model, tokenizer, prompt, response):
    """답변의 생성확률(로그) 계산하는 함수"""
    full_text = prompt + response  # prompt + 답변
    inputs = tokenizer(full_text, return_tensors='pt').to(model.device)  # 토큰화시키고 GPU/CPU로 이동
    prompt_len = len(tokenizer(prompt, return_tensors='pt')['input_ids'][0])  # prompt가 몇 토큰인지 길이
    print(full_text)
    print(len(inputs['input_ids'][0]))  # 응답을 포함한 전체 토큰 길이
    print(prompt_len)  # 프롬프트의 토큰 길이

    # 추론 모드(그래프 미생성 / 계산만 진행) : 메모리 절약
    with torch.no_grad():
        outputs = model(**inputs) # 순전파 (batch_size, seq_len, vocab_size)
        logits = outputs.logits   # 각 토큰 위치별 다음 토큰 로짓
        print(logits.shape)       # 로짓 텐서 형태 확인
        log_probs = F.log_softmax(logits, dim=-1) # 로짓 -> log_prob로 변환(수치 안정)
        print(log_probs.shape)    # log_pob 텐서 형태 확인

        print(f'생성할 토큰의 확률모음: {log_probs[:, :-1].shape}')  # 예측 분포 (마지막 토큰 제외)
        print(f'정답 토큰 id: {inputs['input_ids'][:, 1:].unsqueeze(-1).shape}')  # 다음 토큰 정답 id
        # 생성토큰의 확률추출 (batch, seq_len-1, vocab_size)
        token_log_probs = log_probs[:, :-1].gather(  # log_probs의 마지막 위치는 다음 토큰 예측할게 없으므로 제외
            index=inputs['input_ids'][:, 1:].unsqueeze(-1), # 정답 토큰 id  (batch, seq_len-1, 1)
            dim=-1    # 마지막 차원 (vocab_size)
        ).squeeze(-1)  # (batch, seq_len-1)
        print(f'정답 토큰의 생성확률: {token_log_probs.shape}')  # 토큰별 log_prob 확인

        # response부분만 추출(prompt 제외)
        response_log_probs = token_log_probs[:, prompt_len - 1:]  # prompt 구간은 제외하고, response 구간만 추출
        # 누적 생성확률
        total = response_log_probs.sum()  # response 토큰 log_prob 합

    return total.item()  # float형태로 반환

chosen_prob = get_logprob(model, tokenizer, prompt, chosen)  # chosen 답변의 누적 log_prob 계산
rejected_prob = get_logprob(model, tokenizer, prompt, rejected)  # rejected 답변의 누적 log_prob 계산
print(f'선택된 답변 생성확률: {chosen_prob}')
print(f'거절된 답변 생성확률: {rejected_prob}')

파이썬이 뭐야?파이썬은 배우기 쉽고, 강력한 프로그래밍 언어입니다.
20
6
torch.Size([1, 20, 102400])
torch.Size([1, 20, 102400])
생성할 토큰의 확률모음: torch.Size([1, 19, 102400])
정답 토큰 id: torch.Size([1, 19, 1])
정답 토큰의 생성확률: torch.Size([1, 19])
파이썬이 뭐야?파이썬은 뱀이지~
12
6
torch.Size([1, 12, 102400])
torch.Size([1, 12, 102400])
생성할 토큰의 확률모음: torch.Size([1, 11, 102400])
정답 토큰 id: torch.Size([1, 11, 1])
정답 토큰의 생성확률: torch.Size([1, 11])
선택된 답변 생성확률: -85.5
거절된 답변 생성확률: -59.0


.gather(  
    index=...,    # 정답 토큰 id  
    dim=-1        # 마지막 차원 (vocab_size)  
)   : dim 차원에서 index에 주어진 값을 가져온다.  
=> vocab 차원에서 정답 토큰을 뽑아서 가져온다.

In [ ]:
# dpo 손실함수
# - 정책모델, 참조모델의 차이를 작게 만들려는 목적(손실)함수를 사용
def dpo_loss(policy_chosen, policy_rejected, ref_chosen, ref_rejected, beta=0.1):      # DPO loss 계산 함수
    logits = beta * ((policy_chosen - policy_rejected) - (ref_chosen - ref_rejected))  # policy 선호차 - ref 선호차
    loss = -F.logsigmoid(torch.tensor(logits))  # 선호차가 커질수록 loss가 작아지도록 변환
    return loss.item()  # 파이썬 float 형태로 반환

정책모델 선호응답 확률이 참조(비교) 모델의 선호응답 확률보다 크면 클수록(chosen을 선택 확률이 클수록) 손실값이 계속해서 작아지게 된다.